# Testing with [pytest](https://docs.pytest.org/en/latest/) - part 2

In [ ]:
# Assurons nous que les paquets pytest et ipytest sont installés
# ipytest est requis pour faire tourner pytest dans un notebook Jupyter
import sys
!{sys.executable} -m pip install pytest
!{sys.executable} -m pip install ipytest

import ipytest.magics
import pytest
__file__ = '16_les_tests_partie2.ipynb'

## [`@pytest.fixture`](https://docs.pytest.org/en/latest/fixture.html#pytest-fixtures-explicit-modular-scalable)
Supposons disposer de l'implémentation d'une classe `Personne` que nous souhaitons tester.

In [ ]:
# Cela pourrait se trouver dans un fichier personne.py par exemple
class Personne:
    def __init__(self, prenom, nom, age):
        self.prenom = prenom
        self.nom = nom
        self.age = age
    
    @property
    def nom_complet(self):
        return f'{self.prenom} {self.nom}'
    
    @property
    def comme_dictionnaire(self):
        return {'nom': self.nom_complet, 'age': self.age}
        
    def augmenter_age(self, annees):
        if annees < 0:
            raise ValueError('Je ne peux pas rajeunir les gens :(')
        self.age += annees

Vous pouvez facilement réutiliser du code de test en utilisant les «fixtures» de pytest.
Si vous placez vos fixtures dans [_conftest.py_](https://docs.pytest.org/en/latest/fixture.html#conftest-py-sharing-fixture-functions), ces fixtures sont disponibles pour tout vos scénarios de test.
En général, le fichier _conftest.py_ se situe à la racine de votre répertoire _tests_.

In [ ]:
# Cela serait soit dans conftest.py soit dans test_personne.py
@pytest.fixture()
def personne_par_defaut():
    personne = Personne(prenom='John', nom='Doe', age=82)
    return personne

Dès lors, vous pouvez utiliser la fixture `personne_par_defaut` dans les scénarios de tests effectifs. 

In [ ]:
%%run_pytest[clean]

# Cela se trouverait dans le fichier test_personne.py
def test_nom_complet(personne_par_defaut): # Note: nous utilisons la fixture comme un argument du test
    resultat = personne_par_defaut.nom_complet
    assert resultat == 'John Doe'
    
    
def test_comme_dictionnaire(personne_par_defaut):
    attendu = {'nom': 'John Doe', 'age': 82}
    resultat = personne_par_defaut.comme_dictionnaire
    assert resultat == attendu
    
    
def test_augmenter_age(personne_par_defaut):
    personne_par_defaut.augmenter_age(1)
    assert personne_par_defaut.age == 83
    
    personne_par_defaut.augmenter_age(10)
    assert personne_par_defaut.age == 93
    
    
def test_augmenter_age_avec_nombre_negatif(personne_par_defaut):
    with pytest.raises(ValueError):
        personne_par_defaut.augmenter_age(-1)

En utilisant une fixture, nous pouvons utiliser la même `personne_par_defaut` pour tout nos scénarios de test!

Dans le `test_augmenter_age_avec_nombre_negatif` nous avons utilisé [`pytest.raises`](https://docs.pytest.org/en/latest/assert.html#assertions-about-expected-exceptions) pour vérifier qu'une exception a bien été levée. 

## [`@pytest.mark.parametrize`](https://docs.pytest.org/en/latest/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions)
Parfois vous voulez tester la même fonctionnalité avec de multiples entrées. `pytest.mark.parametrize` est votre solution pour définir de multiples entrées avec les sorties attendues. Considérons l'implémentation suivante de la fonction `remplacer_noms`.

In [ ]:
# Par exemple dans string_manipulate.py
def remplacer_noms(chaine_originale, nouveau_nom):
    """Remplace les noms (qui débutent par une majuscule) de la chaine_originale par nouveau_nom"""
    mots = chaine_originale.split()
    mots_manipulees = [nouveau_nom if mot.istitle() else mot for mot in mots]
    return ' '.join(mots_manipulees)

Nous pouvons tester la fonction `remplacer_noms` avec plusieurs entrées en utilisant `pytest.mark.parametrize`.

In [ ]:
%%run_pytest[clean]

# Cela pourrait être dans votre module de test
@pytest.mark.parametrize("orig,nom,attendu", [
        ('je suis Lisa', 'John Doe', 'je suis John Doe'),
        ('comment vont Anne et Bob', 'John', 'comment vont John et John'),
        ('pas de nom ici', 'John Doe', 'pas de nom ici'),
    ])
def test_remplacer_noms(orig, nom, attendu):
    resultat = remplacer_noms(orig, nom)
    assert resultat == attendu